In [1]:
%load_ext blackcellmagic

In [2]:
# TODO: remove unnecessary libraries
import pandas as pd
import numpy as np
import json
# import openai
# import base64
import os
from langchain import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import datetime

In [4]:
# getting keys
# TODO: remove unnecessary aspects
with open('../config.json') as f:
    keys = json.load(f)
PATH = keys['path']
# openai_organization = keys['openai_organization']
# openai.organization = openai_organization
openai_api_key = keys['openai_api_key']
# openai.api_key = openai_api_key

In [5]:
os.environ["OPENAI_API_KEY"] = openai_api_key

# Variables 

In [6]:
# Get the current date and time
now = datetime.datetime.now()

In [7]:
# Format the date and time as a string in the desired format
timestamp = now.strftime("%Y%m%d%H%M%S")

# Functions

In [8]:
def save_to_txt(name, input_string):
    # Define the file name with the timestamp
    # TODO: add the PATH to the filename
    filename = f"../data/{timestamp}_{name}.txt"
    # Write the string to the file
    with open(filename, "w") as file:
        file.write(input_string)

In [9]:
def create_feedback(code_input):
    # defining the prompt template for a standardized input
    # TO EXPLORE: refine prompt
    feedback_prompt = PromptTemplate(
        input_variables=["code"],
        template="Please review the following code and give five recommendations with detailed explanations how to improve the programming: {code}?",
    )
    
    # initializing the LM
    # TO EXPLORE: adjust temperature
    # TO EXPLORE: test other LMs
    feedback_llm = OpenAI(temperature=0.3, , model_name='gpt-4')
    
    # a simple chain taking user input, formatting the prompt and sending it to the LM
    feedback_chain = LLMChain(llm=feedback_llm, prompt=feedback_prompt)
    
    # Run the chain only specifying the input variable.
    feedback = feedback_chain.run(code_input)
    
    # saving the input to txt via the prepared function
    save_to_txt(name='feedback', input_string=feedback)
    
    return feedback

In [10]:
def shorten_feedback(feedback):
    # defining the prompt template for a standardized input
    short_feedback_prompt = PromptTemplate(
        input_variables=["feedback"],
        template="Please shorten the aspects of the following feeback: {feedback}?",
    )
    
    # initializing the LM
    # TO OPTIMIZE
    short_feedback_llm = OpenAI(temperature=0)
    
    # a simple chain taking user input, formatting the prompt and sending it to the LM
    short_feedback_chain = LLMChain(llm=short_feedback_llm, prompt=short_feedback_prompt)
    
    # Run the chain only specifying the input variable.
    short_feedback = short_feedback_chain.run(feedback)
    
    # saving the input to txt via the prepared function
    save_to_txt(name='shortfeedback', input_string=short_feedback)
    
    return short_feedback

In [11]:
# TODO: separate into a reading and a merging function and reuse reading function
def pick_shortfeedbacks(directory, n):
    # TODO: use PATH here for directory
    
    # Get a list of all files in the directory
    files = os.listdir(directory)
    
    # Filter the list to only include files with the correct format
    files = [f for f in files if f.endswith("_shortfeedback.txt") and len(f) == 32]
    
    # Sort the list of files by date, with the most recent file first
    files.sort(reverse=True)
    
    # Get the three most recent files
    latest_files = files[:n]
    
    # Read the contents of the two files into string variables
    file_contents = []
    for file in latest_files:
        with open(os.path.join(directory, file), "r") as f:
            file_contents.append(f.read())
            
    # Combine the two file contents into a single string variable
    latest_short_feedbacks = "\n".join(file_contents)
    
    return latest_short_feedbacks

In [12]:
# TODO: improve the template without making it break
def define_goal(latest_short_feedbacks):
    
    # defining the prompt template for a standardized input
    learning_goal_prompt = PromptTemplate(
        input_variables=["short_feedback"],
        template="Summarize the following points: {short_feedback}",
    )
    
    # initializing the LM
    # TODO: check for optimal LLM
    learning_goal_llm = OpenAI(temperature=0.5)
    
    # a simple chain taking user input, formatting the prompt and sending it to the LM
    learning_goal_chain = LLMChain(llm=learning_goal_llm, prompt=learning_goal_prompt)
    
    # Run the chain only specifying the input variable.
    learning_goals = learning_goal_chain.run(latest_short_feedbacks)
    
    # saving the input to txt via the prepared function
    save_to_txt(name='learninggoals', input_string=learning_goals)
    
    return learning_goals

In [13]:
# TODO: what happens if there is no learning goal?
# it is supposed to not produce an error but only return something empty
def evaluate_code(directory, code_input):
    # Get a list of all files in the directory
    files = os.listdir(directory)

    # Filter the list to only include files with the correct format
    files = [f for f in files if f.endswith("_learninggoals.txt") and len(f) == 32]

    # Sort the list of files by date, with the most recent file first
    files.sort(reverse=True)

    # Get the most recent file
    latest_file = files[:1]

    with open(os.path.join(directory, latest_file[0]), "r") as f:
        learning_goals = f.read()
    
    # defining the promp template to get both the latest code input and learning_goals
    evaluation_prompt = PromptTemplate(
        input_variables=["code_input", "learning_goals"],
        template="Please compare this code: {code_input} with these learning goals: {learning_goals}. If the programmer considered the learning goals when writing the provided code, say something motivating. If the programmer didn't consider the learning goals, gently remind the person of their learning goals.",
    )
    
    # application of the template
    evaluation_prompt = evaluation_prompt.format(code_input=code_input, learning_goals=learning_goals)
    
    # llm definition
    evaluation_llm = OpenAI(temperature=0.3)
    
    evaluation = evaluation_llm(evaluation_prompt)
    
    return evaluation

In [14]:
def get_feedback(code_input, directory):
    
    # create feedback from input
    feedback = create_feedback(code_input=code_input)
    
    # stop the program if the created output is empty
    assert len(feedback) != 0, 'The created feedback was empty'
    
    # create short feedback for goal definition
    short_feedback = shorten_feedback(feedback=feedback)

    # stop the program if the created output is empty
    assert len(short_feedback) != 0, 'The created shortfeedback was empty'
    
    # evaluate if the learning goals have been applied while coding
    evaluation = evaluate_code(directory=directory, code_input=code_input)
    
    return feedback, evaluation

# Code review

In [8]:
# feature to be added: read from Github

In [15]:
# add code input as text here
code_input = str('''
def save_to_txt(name, input_string, timestamp):
    # Define the file name with the timestamp
    # TODO: add the PATH to the filename
    filename = f"../data/{timestamp}_{name}.txt"
    # Write the string to the file
    with open(filename, "w") as file:
        file.write(input_string) 
        '''
)

In [16]:
# saving the input to txt via the prepared function
save_to_txt(name='codeinput', input_string=code_input)

In [17]:
%%time
# conditional execution of code_input was passed
if code_input is not None:
    
    feedback, evaluation = get_feedback(code_input=code_input, directory='../data')
    
    print(feedback)
    print(evaluation)



1. Move the functions outside of the main code block. This will make the code easier to read and debug. 
2. Add comments to explain what each function does. This will help other developers understand the code. 
3. Use descriptive variable names. This will make the code easier to read and understand. 
4. Use more descriptive names for the functions. This will make the code easier to read and understand. 
5. Add error handling for invalid inputs. This will help ensure that the code runs correctly and produces the expected results.


The programmer appears to have considered the learning goals when writing the provided code. The code is organized into separate functions, uses descriptive variable names, and consistent indentation. Additionally, the code is commented to explain the code and a linter is used to check for errors. Well done!
CPU times: user 71.2 ms, sys: 17.9 ms, total: 89.1 ms
Wall time: 14.5 s


# Set learning goals

In [18]:
# This has to be done once and then only if the user checks a button like "generate learning targets".

In [19]:
# currently pseudo-button:
button = True

In [20]:
%%time
if button:
    # get latest short feedbacks
    latest_short_feedbacks = pick_shortfeedbacks(directory='../data', n=3)
    # define goal from latest short feedbacks
    learning_goals = define_goal(latest_short_feedbacks=latest_short_feedbacks)
    
    # stop the program if the created output is empty
    assert len(learning_goals) != 0, 'The created learning goal was empty'
    
    print('Your new goals are:' + learning_goals)

Your new goals are:

Organize code into separate functions to improve readability and maintainability, use descriptive variable and function names, and include comments to explain code and error handling for invalid inputs.
CPU times: user 12.4 ms, sys: 4.51 ms, total: 16.9 ms
Wall time: 2.91 s


# TO INCLUDE

### one extensive function

In [16]:
%%time
# somehow doesn't work
learning_goals = define_goal(latest_short_feedbacks=latest_short_feedbacks)

CPU times: user 53.1 ms, sys: 11.6 ms, total: 64.7 ms
Wall time: 693 ms


In [17]:
learning_goals

''

### extensive step by step

### short input step by step